In [6]:
#Task 10: Embeddings , Vector DBs
#Using RAG , Build an e-commerce assistant that provides intelligent recommendations and answer user's queries about the products.

from langchain_ollama import ChatOllama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document

# -------------------------------------------------
#  Load product data 
# -------------------------------------------------
product_data = """
Product: Google Pixel 8
Category: Smartphone
Price: ₹55,000
Description: Google’s latest smartphone with Tensor G3 chip and excellent camera performance.

Product: Bose QuietComfort 45
Category: Headphones
Price: ₹28,000
Description: Premium noise-cancelling headphones with superior comfort and sound quality.

Product: HP Spectre x360
Category: Laptop
Price: ₹1,10,000
Description: Convertible laptop with Intel i7 processor, touchscreen, and sleek design.

Product: Samsung Galaxy Tab S9
Category: Tablet
Price: ₹60,000
Description: High-end Android tablet with AMOLED display and stylus support.

Product: JBL Flip 6
Category: Speakers
Price: ₹10,000
Description: Portable Bluetooth speaker with powerful bass and waterproof design.
"""

# Wrap product data into a LangChain Document
documents = [Document(page_content=product_data)]

# -------------------------------------------------
#  Chunk the data
# -------------------------------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

# -------------------------------------------------
# Create embeddings (open-source)
# -------------------------------------------------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# -------------------------------------------------
# Store embeddings in FAISS
# -------------------------------------------------
vectorstore = FAISS.from_documents(
    chunks,
    embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

# -------------------------------------------------
# Create LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)

# -------------------------------------------------
# RAG Prompt
# -------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an e-commerce assistant. "
            "Use the given context to answer the question."
        ),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}")
    ]
)

# -------------------------------------------------
#  RAG Pipeline
# -------------------------------------------------
def ask_question(question: str):
    docs = retriever.invoke(question)
    context = "\n\n".join(doc.page_content for doc in docs)

    chain = prompt | llm
    response = chain.invoke(
        {
            "context": context,
            "question": question
        }
    )
    return response.content

# -------------------------------------------------
#  Test queries
# -------------------------------------------------
if __name__ == "__main__":
    questions = [
        "Recommend a good smartphone under 70000",
        "Which laptop is best for portability?",
        "Suggest noise cancelling headphones",
    ]

    for q in questions:
        print(f"\nQuestion: {q}")
        answer = ask_question(q)
        print("Answer:", answer)


Question: Recommend a good smartphone under 70000
Answer: Based on the given products, I would recommend the Google Pixel 8. It's a high-end smartphone with excellent camera performance and a Tensor G3 chip, all for a price of ₹55,000 which is well within your budget of ₹70,000.

Question: Which laptop is best for portability?
Answer: Based on the given products, I would recommend the HP Spectre x360 as the best option for portability. As a convertible laptop, it offers the flexibility to be used in various modes (laptop, tablet, tent, and stand) making it an excellent choice for those who need a device that can adapt to different situations. Its sleek design also makes it lightweight and easy to carry around.

Question: Suggest noise cancelling headphones
Answer: Based on the given context, I would suggest the "Bose QuietComfort 45" as a great option for noise-cancelling headphones. It's a premium product with superior comfort and sound quality, priced at ₹28,000.
